# Experiment PAMAP2 with mcfly

This experiment finds an optimal model for the PAMAP2 dataset.

## Import required Python modules

In [3]:
import sys
import os
sys.path.insert(0, os.path.abspath('../..'))
import numpy as np
import pandas as pd
# mcfly
from mcfly import tutorial_pamap2, modelgen, find_architecture, storage
# Keras module is use for the deep learning
import keras
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Activation, Convolution1D, Flatten, MaxPooling1D
from keras.optimizers import Adam
# We can set some backend options to avoid NaNs
from keras import backend as K

## Load the data

In [4]:
datapath = '/media/sf_VBox_Shared/timeseries/PAMAP2_Dataset/slidingwindow512cleaned/'
Xs = []
ys = []

ext = '.npy'
for i in range(9):
    Xs.append(np.load(datapath+'X_train_'+str(i)+ext))
    ys.append(np.load(datapath+'y_train_binary'+str(i)+ext))

## Generate models

First step is to create a model architecture. As we do not know what architecture is best for our data we will create a set of models to investigate which architecture is most suitable for our data and classification task. You will need to specificy how many models you want to create with argument 'number_of_models', the type of model which can been 'CNN' or 'DeepConvLSTM', and maximum number of layers per modeltype. See for a full overview of the optional arguments the function documentation of modelgen.generate_models

In [5]:
num_classes = ys[0].shape[1]
np.random.seed(123)
models = modelgen.generate_models(Xs[0].shape,
                                  number_of_classes=num_classes,
                                  number_of_models = 15)

In [6]:
models[0][1]

{'filters': array([27, 93, 67, 96, 57, 83, 42]),
 'learning_rate': 0.013854217299751215,
 'lstm_dims': array([35]),
 'regularization_rate': 0.02086630923723395}

## Compare models
Now that the model architectures have been generated it is time to compare the models by training them in a subset of the training data and evaluating the models in the validation subset. This will help us to choose the best candidate model. Performance results are stored in a json file.

In [7]:
# Define directory where the results, e.g. json file, will be stored
resultpath = '/media/sf_VBox_Shared/timeseries/PAMAP2_Dataset/results/' 
if not os.path.exists(resultpath):
        os.makedirs(resultpath)

In [8]:
def split_train_test(X_list, y_list, j):
    X_train = np.concatenate(X_list[0:j]+X_list[j+1:])
    X_test = X_list[j]
    y_train = np.concatenate(y_list[0:j]+y_list[j+1:])
    y_test = y_list[j]
    return X_train, y_train, X_test, y_test

def split_train_small_val(X_list, y_list, j, trainsize=500, valsize=500):
    X = np.concatenate(X_list[0:j]+X_list[j+1:])
    y = np.concatenate(y_list[0:j]+y_list[j+1:])
    rand_ind = np.random.choice(X.shape[0], trainsize+valsize, replace=False)
    X_train = X[rand_ind[:trainsize]]
    y_train = y[rand_ind[:trainsize]]
    X_val = X[rand_ind[trainsize:]]
    y_val = y[rand_ind[trainsize:]]
    return X_train, y_train, X_val, y_val

In [9]:
from keras.models import model_from_json

def get_fresh_copy(model, lr):
    model_json = model.to_json()
    model_copy = model_from_json(model_json)
    model_copy.compile(loss='categorical_crossentropy',
                  optimizer=Adam(lr=lr),
                  metrics=['accuracy'])
    #for layer in model_copy.layers:
    #    layer.build(layer.input_shape)
    return model_copy

In [8]:
models = [(get_fresh_copy(model, params['learning_rate']), params, model_type)  for model, params, model_type in models]

In [9]:
import time
t = time.time()
np.random.seed(123)
histories_list, val_accuracies_list, val_losses_list = [], [], []
for j in range(8, len(Xs)):
    print('fold '+str(j))
    models = [(get_fresh_copy(model, params['learning_rate']), params, model_type)  for model, params, model_type in models]
    X_train, y_train, X_val, y_val = split_train_small_val(Xs, ys, j, trainsize=500, valsize=500)
    histories, val_accuracies, val_losses = find_architecture.train_models_on_samples(X_train, y_train,
                                                                           X_val, y_val,
                                                                           models,
                                                                           nr_epochs=10,
                                                                           subset_size=500,
                                                                           verbose=True,
                                                                           outputfile=resultpath+\
                                                                                  'experiment'+str(j)+'.json',
                                                                           early_stopping=True)
    histories_list.append(histories)
    val_accuracies_list.append(val_accuracies)
    val_losses.append(val_losses)
print(time.time()-t)

fold 8
Training model 0 DeepConvLSTM
Train on 500 samples, validate on 500 samples
Epoch 1/10
500/500 [==============================] - 156s - loss: 4.8019 - acc: 0.1540 - val_loss: 2.5160 - val_acc: 0.1000
Epoch 2/10
500/500 [==============================] - 148s - loss: 3.6550 - acc: 0.1300 - val_loss: 2.3931 - val_acc: 0.1200
Epoch 3/10
500/500 [==============================] - 158s - loss: 2.7197 - acc: 0.1860 - val_loss: 2.4210 - val_acc: 0.1840
Training model 1 DeepConvLSTM
Train on 500 samples, validate on 500 samples
Epoch 1/10
500/500 [==============================] - 119s - loss: 3.1419 - acc: 0.0940 - val_loss: 2.4291 - val_acc: 0.1200
Epoch 2/10
500/500 [==============================] - 116s - loss: 2.6125 - acc: 0.1100 - val_loss: 2.4442 - val_acc: 0.0760
Training model 2 DeepConvLSTM
Train on 500 samples, validate on 500 samples
Epoch 1/10
500/500 [==============================] - 112s - loss: 3.2211 - acc: 0.1000 - val_loss: 2.5904 - val_acc: 0.0600
Epoch 2/10
500/

In [10]:
# Read them all back in
import json
model_jsons = []
for j in range(len(Xs)):
    with open(resultpath+'experiment'+str(j)+'.json', 'r') as outfile:
        model_jsons.append(json.load(outfile))
model_jsons[0] = model_jsons[0][-15:]

In [11]:
model_jsons[0][0].keys()

dict_keys(['regularization_rate', 'val_acc', 'modeltype', 'lstm_dims', 'learning_rate', 'train_loss', 'train_acc', 'val_loss', 'filters'])

In [12]:
val_accuracies = np.array([[mod['val_acc'][-1] for mod in fold] for fold in model_jsons])
[print(len(a)) for a in val_accuracies];

15
15
15
15
15
15
15
15
15


In [13]:
val_acc = np.array([np.array([mod['val_acc'][-1] for mod in fold], dtype='float') for fold in model_jsons])
train_acc = np.array([np.array([mod['train_acc'][-1] for mod in fold], dtype='float') for fold in model_jsons])
train_loss = np.array([np.array([mod['train_loss'][-1] for mod in fold], dtype='float') for fold in model_jsons])
val_loss = np.array([np.array([mod['val_loss'][-1] for mod in fold], dtype='float') for fold in model_jsons])

In [14]:
val_accuracies_avg = val_acc.mean(axis=0)
val_accuracies_avg

array([ 0.17644445,  0.136     ,  0.14977778,  0.77866667,  0.77911111,
        0.39733334,  0.20022222,  0.11422222,  0.60088889,  0.74888889,
        0.75511112,  0.81466667,  0.18844445,  0.10666667,  0.76155556])

In [ ]:
# train_acc = np.array([[history.history['acc'][-1] for history in histories] for histories in histories_list])
# train_loss = np.array([[history.history['loss'][-1] for history in histories] for histories in histories_list])
# val_acc = np.array([[history.history['val_acc'][-1] for history in histories] for histories in histories_list])
# val_loss = np.array([[history.history['val_loss'][-1] for history in histories] for histories in histories_list])

Another way of comparing model performance is by putting all the information in a pandas dataframe, which we can store in a csv file.

In [15]:
modelcomparisons = pd.DataFrame({'model':[str(params) for model, params, model_types in models],
                       'train_acc': train_acc.mean(axis=0),
                       'train_loss': train_loss.mean(axis=0),
                       'val_acc': val_acc.mean(axis=0),
                       'val_loss': val_loss.mean(axis=0)
                       })
modelcomparisons

,model,train_acc,train_loss,val_acc,val_loss
0,"{'regularization_rate': 0.02086630923723395, '...",0.264000,2.898308,0.176444,2.403033
1,"{'regularization_rate': 0.0022288077746968685,...",0.131556,2.698856,0.136000,2.393217
2,{'regularization_rate': 0.00017892532817003035...,0.170000,2.613123,0.149778,2.409913
3,"{'regularization_rate': 0.0017941745604499644,...",0.834000,2.186001,0.778667,0.844342
4,"{'regularization_rate': 0.001089490983523001, ...",0.927556,1.244737,0.779111,0.992753
5,"{'regularization_rate': 0.006174523760141131, ...",0.585111,8.875916,0.397333,2.180829
6,"{'regularization_rate': 0.07596584041274666, '...",0.229556,3.538028,0.200222,2.270040
7,"{'regularization_rate': 0.0967007204303304, 'l...",0.122667,2.769499,0.114222,2.484940
8,"{'regularization_rate': 0.016069244803607875, ...",0.674889,3.624188,0.600889,1.283478
9,"{'regularization_rate': 0.025948092548121634, ...",0.681778,11.305308,0.748889,1.073372


It is also possible to vizualize the performance of the various models using our vizualisation tool as explained in the mcfly repository README file: https://github.com/NLeSC/mcfly/blob/master/README.md

Check which model is the best

In [17]:
best_model_index = np.argmax(val_accuracies_avg)
best_model, best_params, best_model_types = models[best_model_index]
print('Model type and parameters of the best model:')
print(best_model_types)
print(best_params)

Model type and parameters of the best model:
CNN
{'regularization_rate': 0.00029035321282643397, 'fc_hidden_nodes': 196, 'learning_rate': 0.006326663929636657, 'filters': array([35, 16, 64])}


In [60]:
modelname = 'bestmodel_sample'
storage.savemodel(best_model,resultpath,modelname)

('/media/sf_VBox_Shared/timeseries/PAMAP2_Dataset/results/bestmodel_sample_architecture.json',
 '/media/sf_VBox_Shared/timeseries/PAMAP2_Dataset/results/bestmodel_sample_weights')

## Train the best model for real

Now that we have identified the best model architecture out of our random pool of models we can continue by training the model on the full training sample. For the purpose of speeding up the example we only train the full model on the first 1000 values. You will need to replace this by 'datasize = X_train.shape[0]' in a real world example.

In [14]:
len(Xs)

9

In [15]:
nr_epochs = 2

np.random.seed(123)
histories, test_accuracies_list, models = [], [], []
for j in range(len(Xs)):
    X_train, y_train, X_test, y_test = split_train_test(Xs, ys, j)
    model_copy = get_fresh_copy(best_model, best_params['learning_rate'])
    datasize = X_train.shape[0]
    
    history = model_copy.fit(X_train[:datasize,:,:], y_train[:datasize,:],
              nb_epoch=nr_epochs, validation_data=(X_test, y_test))
    
    histories.append(history)
    test_accuracies_list.append(history.history['val_acc'][-1] )
    models.append(model_copy)

Train on 14663 samples, validate on 2155 samples
Epoch 1/2
14663/14663 [==============================] - 289s - loss: 1.3279 - acc: 0.8916 - val_loss: 0.9985 - val_acc: 0.8418
Epoch 2/2
14663/14663 [==============================] - 300s - loss: 0.4954 - acc: 0.9351 - val_loss: 1.0915 - val_acc: 0.7968
Train on 14528 samples, validate on 2290 samples
Epoch 1/2
14528/14528 [==============================] - 286s - loss: 1.3542 - acc: 0.8890 - val_loss: 1.2473 - val_acc: 0.5581
Epoch 2/2
14528/14528 [==============================] - 301s - loss: 0.5336 - acc: 0.9257 - val_loss: 1.2055 - val_acc: 0.5934
Train on 15344 samples, validate on 1474 samples
Epoch 1/2
15344/15344 [==============================] - 343s - loss: 1.3298 - acc: 0.8914 - val_loss: 0.3195 - val_acc: 0.9166
Epoch 2/2
15344/15344 [==============================] - 355s - loss: 0.5002 - acc: 0.9337 - val_loss: 0.6245 - val_acc: 0.8080
Train on 14799 samples, validate on 2019 samples
Epoch 1/2
14799/14799 [=============

In [19]:
print(np.mean(test_accuracies_list))
test_accuracies_list

0.771778561345


[0.79675174027750495,
 0.59344978186761443,
 0.80800542740841252,
 0.55968301140653909,
 0.87647058833546998,
 0.9288664525011473,
 0.89486796237012312,
 0.48791208793828778,
 1.0]

In [21]:
test_accuracies_list = [0.79675174027750495,
 0.59344978186761443,
 0.80800542740841252,
 0.55968301140653909,
 0.87647058833546998,
 0.9288664525011473,
 0.89486796237012312,
 0.48791208793828778,
 1.0]

In [19]:
# Calculate 1-NN for each fold:
nr_epochs = 2

np.random.seed(123)
knn_test_accuracies_list = []
for j in range(len(Xs)):
    print("fold ", j)
    X_train, y_train, X_test, y_test = split_train_test(Xs, ys, j)
    acc = find_architecture.kNN_accuracy(X_train, y_train, X_test, y_test, k=1)
    knn_test_accuracies_list.append(acc )

fold  0
fold  1
fold  2
fold  3
fold  4
fold  5
fold  6
fold  7
fold  8


In [22]:
print(np.mean(knn_test_accuracies_list))
accs_compared = pd.DataFrame({'CNN': test_accuracies_list, 'kNN':knn_test_accuracies_list})
accs_compared

0.53974709837


,CNN,kNN
0,0.796752,0.611601
1,0.593450,0.610044
2,0.808005,0.613976
3,0.559683,0.523031
4,0.876471,0.615966
5,0.928866,0.523176
6,0.894868,0.603886
7,0.487912,0.371429
8,1.000000,0.384615


### Saving, loading and comparing reloaded model with orignal model

The modoel can be saved for future use. The savemodel function will save two separate files: a json file for the architecture and a npy (numpy array) file for the weights.

In [16]:
modelname = 'my_bestmodel'

In [18]:
for i, model in enumerate(models):
    storage.savemodel(model,resultpath,modelname+str(i))